In [10]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import  StandardScaler
# from sklearn.decomposition import PCA

# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold

# import keras
from keras import models, layers #, callbacks, constraints, backend, activations
from keras.models import model_from_json
from sklearn.utils import resample


# from matplotlib import pyplot
# import matplotlib.patches as mpatches

# import datetime
import pathlib
import time
# import json
# import sys
import os

# %matplotlib inline
# pd.options.display.max_columns = None

from Utility import scale, ann_model, string_time, ensembleModels

## load datasets

In [2]:
train_dataset = pd.DataFrame.from_csv('../dataset/train.csv', index_col=None)
test_dataset = pd.DataFrame.from_csv('../dataset/test.csv', index_col=None)

/home/luis/anaconda3/envs/tf-source-keras/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  if __name__ == '__main__':
/home/luis/anaconda3/envs/tf-source-keras/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  from ipykernel import kernelapp as app


## split training into X_train, y_train & rename train_dataset into X_test

In [3]:
y_train = pd.get_dummies(train_dataset['Cover_Type'],  prefix='Cover_Type_')
y_single = train_dataset['Cover_Type']
X_train = train_dataset.drop('Cover_Type', axis=1)
X_train.shape, y_train.shape, y_single.shape

((15120, 55), (15120, 7), (15120,))

In [4]:
X_test = test_dataset.values
X_test.shape

(565892, 55)

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

/home/luis/anaconda3/envs/tf-source-keras/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


### load models

In [6]:
model_number = set()
models = []

filepath = '../models'

for file in os.listdir(filepath):
    
    if file == '.ipynb_checkpoints':
        continue
    
    index_start = file.find('_')
    index_end = file.find('.')

    model_number.add(file[index_start + 1: index_end])
    
    

for number in model_number:
  
    json_file = open('{}/model_{}.json'.format(filepath, number), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(filepath= '{}/model_{}.hdf5'.format(filepath, number))
    
    models.append(model)

### make ensemble classifier

In [7]:
model_input = layers.Input(shape=models[0].input_shape[1:])
ens = ensembleModels(models, model_input)

### predict on training data

In [11]:
predictions = ens.predict(X_train_scaled)
y_hat = predictions.argmax(axis=1) + 1
y = y_train.values.argmax(axis=1) + 1

accuracy_score(y, y_hat)

0.9137566137566138

### make predictions for testing data and save for kaggle submittion

In [21]:
predictions = ens.predict(X_test_scaled)
y_hat = predictions.argmax(axis=1) + 1

In [22]:
results = test_dataset[['Id']]
results['Cover_Type'] = y_hat

/home/luis/anaconda3/envs/tf-source-keras/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [24]:
results.to_csv('submission.csv', index=False)